# GREAT CARIA FINAL - Systemic Fragility Index

## Simplified Model (Back to Basics)

After extensive experimentation, this is the **definitive model** that maximizes signal while minimizing complexity.

### Core Components:
1. **Crisis Factor (CF)** - Correlation × Volatility (AUC=0.705)
2. **3-Band Decomposition** - Slow/Medium/Fast (not 5)
3. **Factor Analysis** - Latent fragility F_t
4. **Cusp Bifurcation** - Catastrophe theory for transitions

### What was removed:
- ❌ 5 temporal scales (overfitting)
- ❌ Psychology as input (emergent, not cause)
- ❌ Complex physics-first weights
- ❌ Multi-scale Kuramoto

In [ ]:
# === SETUP ===
!pip install PyWavelets scikit-learn statsmodels -q

import pandas as pd
import numpy as np
from scipy import stats, signal
from scipy.ndimage import gaussian_filter1d
import pywt
from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

print('=== GREAT CARIA FINAL ===')
print('Simplified Model: CF + 3 Bands + Factor Analysis + Cusp')

In [ ]:
# === LOAD DATA ===
DATA_PATH = '/content/drive/MyDrive/Caria/yahoo_market.parquet'
df = pd.read_parquet(DATA_PATH)
df.index = pd.to_datetime(df.index)

COUNTRIES = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'FRA', 'BRA', 'MEX', 'KOR', 'AUS', 'IND', 'ZAF']
idx_cols = [f'{c}_index' for c in COUNTRIES if f'{c}_index' in df.columns]
ret = df[idx_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]

print(f'Data: {ret.shape[0]} days, {ret.shape[1]} countries')
print(f'Period: {ret.index.min().date()} to {ret.index.max().date()}')

In [ ]:
# === CRISIS CATALOG (13 events) ===
CRISES = {
    'DotCom': pd.Timestamp('2000-03-10'),
    '9/11': pd.Timestamp('2001-09-11'),
    'Lehman': pd.Timestamp('2008-09-15'),
    'Flash_Crash': pd.Timestamp('2010-05-06'),
    'Euro_Crisis': pd.Timestamp('2011-08-05'),
    'Taper_Tantrum': pd.Timestamp('2013-05-22'),
    'China_Crash': pd.Timestamp('2015-08-24'),
    'Brexit': pd.Timestamp('2016-06-24'),
    'Volmageddon': pd.Timestamp('2018-02-05'),
    'Repo_Crisis': pd.Timestamp('2019-09-17'),
    'COVID': pd.Timestamp('2020-03-11'),
    'Gilt_Crisis': pd.Timestamp('2022-09-23'),
    'SVB': pd.Timestamp('2023-03-10')
}

# Filter to available data
CRISES = {k: v for k, v in CRISES.items() if v >= ret.index.min()}
print(f'\nCrises in data range: {len(CRISES)}')
for name, date in CRISES.items():
    print(f'  {name}: {date.date()}')

---
# PART 1: CRISIS FACTOR (CF)

The **star performer** (AUC = 0.705)

$$CF_t = \bar{\rho}_t \times \bar{\sigma}_t \times 100$$

Where:
- $\bar{\rho}_t$ = average pairwise correlation in window
- $\bar{\sigma}_t$ = average volatility in window

In [ ]:
# === CRISIS FACTOR ===
def compute_cf(r, window=20):
    """Crisis Factor = avg_correlation × avg_volatility"""
    cf = []
    for i in range(window, len(r)):
        wr = r.iloc[i-window:i]
        corr_matrix = wr.corr().values
        n = len(corr_matrix)
        avg_corr = (corr_matrix.sum() - n) / (n * (n - 1))  # Off-diagonal mean
        avg_vol = wr.std().mean()
        cf.append(avg_corr * avg_vol * 100)
    return pd.Series(cf, index=r.index[window:])

CF = compute_cf(ret)
print(f'Crisis Factor computed: {len(CF)} observations')
print(f'CF range: [{CF.min():.3f}, {CF.max():.3f}]')

---
# PART 2: ADDITIONAL SIGNALS FOR FACTOR ANALYSIS

In [ ]:
# === SYNCHRONIZATION (Simple Kuramoto) ===
def compute_sync(returns, window=60):
    """Kuramoto order parameter - phase synchronization"""
    # Extract phases via Hilbert transform
    phases = returns.apply(lambda x: np.angle(signal.hilbert(x.fillna(0).values)))
    
    sync = []
    for i in range(window, len(phases)):
        ph = phases.iloc[i].values
        r = np.abs(np.exp(1j * ph).mean())  # Order parameter
        sync.append(r)
    return pd.Series(sync, index=returns.index[window:])

SYNC = compute_sync(ret)
print(f'Synchronization: mean={SYNC.mean():.3f}')

In [ ]:
# === EARLY WARNING SIGNALS (EWS) ===
def compute_ews(series, window=120):
    """Critical slowing down indicators"""
    acf1 = series.rolling(window).apply(lambda x: x.autocorr(1), raw=False)
    var = series.rolling(window).var()
    skew = series.rolling(window).skew()
    return pd.DataFrame({'acf1': acf1, 'var': var, 'skew': skew})

EWS = compute_ews(CF)
print(f'EWS computed: ACF1, Variance, Skewness')

In [ ]:
# === NETWORK CURVATURE PROXY ===
def rolling_avg_corr(r, window=60):
    """Average correlation as curvature proxy"""
    corrs = []
    for i in range(window, len(r)):
        c = r.iloc[i-window:i].corr().values
        n = len(c)
        avg = (c.sum() - n) / (n * (n - 1))
        corrs.append(avg)
    return pd.Series(corrs, index=r.index[window:])

# Low correlation = low curvature = fragile
CURV = 1 - rolling_avg_corr(ret, 60)
print(f'Curvature proxy computed')

---
# PART 3: FACTOR ANALYSIS → LATENT F_t

Extract a single latent factor that captures common fragility

In [ ]:
# === ALIGN ALL SIGNALS ===
common_idx = CF.index
for sig in [SYNC, EWS['acf1'], EWS['var'], EWS['skew'], CURV]:
    common_idx = common_idx.intersection(sig.dropna().index)

signals_df = pd.DataFrame({
    'cf': CF.loc[common_idx],
    'sync': SYNC.loc[common_idx],
    'acf1': EWS['acf1'].loc[common_idx],
    'var': EWS['var'].loc[common_idx],
    'skew': EWS['skew'].abs().loc[common_idx],
    'curv': CURV.loc[common_idx]
}).dropna()

print(f'Aligned signals: {signals_df.shape}')

In [ ]:
# === FACTOR ANALYSIS ===
print('=== Factor Analysis for Latent Fragility F_t ===')

scaler = StandardScaler()
X = scaler.fit_transform(signals_df)

# Extract single latent factor
fa = FactorAnalysis(n_components=1, random_state=42)
F_latent = fa.fit_transform(X).flatten()
F_t = pd.Series(F_latent, index=signals_df.index, name='F_t')

# Factor loadings
loadings = pd.DataFrame({
    'signal': signals_df.columns,
    'loading': fa.components_[0]
}).sort_values('loading', ascending=False)

print('\nFactor Loadings (contribution to latent fragility):')
print(loadings.to_string(index=False))

# Explained variance
pca = PCA(n_components=1)
pca.fit(X)
print(f'\nVariance explained by F_t: {pca.explained_variance_ratio_[0]:.1%}')

---
# PART 4: 3-BAND TEMPORAL DECOMPOSITION

Only 3 bands (not 5!):
- **Slow** (>60d): Structural/Cycle
- **Medium** (10-60d): Systemic Resonance
- **Fast** (<10d): Critical Noise

In [ ]:
# === WAVELET DECOMPOSITION ===
def decompose_3bands(series, wavelet='db4'):
    """Decompose into slow/medium/fast bands"""
    coeffs = pywt.wavedec(series.values, wavelet, level=6)
    
    # Fast (levels 0-2): ~1-8 days
    fast_coeffs = [np.zeros_like(c) for c in coeffs]
    for i in range(min(3, len(coeffs))):
        fast_coeffs[i] = coeffs[i]
    fast = pywt.waverec(fast_coeffs, wavelet)[:len(series)]
    
    # Medium (levels 3-4): ~8-64 days
    med_coeffs = [np.zeros_like(c) for c in coeffs]
    for i in range(3, min(5, len(coeffs))):
        med_coeffs[i] = coeffs[i]
    medium = pywt.waverec(med_coeffs, wavelet)[:len(series)]
    
    # Slow (levels 5+): >64 days
    slow_coeffs = [np.zeros_like(c) for c in coeffs]
    for i in range(5, len(coeffs)):
        slow_coeffs[i] = coeffs[i]
    slow = pywt.waverec(slow_coeffs, wavelet)[:len(series)]
    
    return {
        'fast': pd.Series(fast, index=series.index),
        'medium': pd.Series(medium, index=series.index),
        'slow': pd.Series(slow, index=series.index)
    }

F_bands = decompose_3bands(F_t)
print('3-Band decomposition complete: Slow, Medium, Fast')

In [ ]:
# === VISUALIZE BANDS ===
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

axes[0].plot(F_t.index, F_t.values, 'k-', alpha=0.7)
axes[0].set_ylabel('F_t (full)')
axes[0].set_title('Latent Fragility and 3-Band Decomposition')

axes[1].fill_between(F_bands['slow'].index, F_bands['slow'].values, alpha=0.5, color='blue')
axes[1].set_ylabel('Slow (>60d)')

axes[2].fill_between(F_bands['medium'].index, F_bands['medium'].values, alpha=0.5, color='orange')
axes[2].set_ylabel('Medium (10-60d)')

axes[3].fill_between(F_bands['fast'].index, F_bands['fast'].values, alpha=0.5, color='red')
axes[3].set_ylabel('Fast (<10d)')

for ax in axes:
    for name, date in CRISES.items():
        if date > F_t.index.min() and date < F_t.index.max():
            ax.axvline(date, color='red', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_final_bands.png', dpi=150)
plt.show()

---
# PART 5: CUSP BIFURCATION DETECTION

In [ ]:
# === CUSP CATASTROPHE PARAMETERS ===
def fit_cusp_params(series, window=120):
    """Estimate cusp parameters from distribution moments"""
    a_params = []
    b_params = []
    bimodality = []
    dates = []
    
    for i in range(window, len(series), 10):
        window_data = series.iloc[i-window:i].values
        
        skew = stats.skew(window_data)
        kurt = stats.kurtosis(window_data)
        
        # Bimodality coefficient (Sarle's)
        bimod = (skew**2 + 1) / (kurt + 3 + 1e-8)
        
        # Map to cusp parameters
        a = -np.sign(kurt) * np.abs(kurt) / 10  # Negative a = bistable
        b = skew / 5
        
        a_params.append(a)
        b_params.append(b)
        bimodality.append(bimod)
        dates.append(series.index[i])
    
    return pd.DataFrame({
        'a': a_params,
        'b': b_params,
        'bimodality': bimodality
    }, index=dates)

cusp = fit_cusp_params(F_t)

# Bifurcation risk: high when a < 0 and variance is high
BIF_RISK = ((cusp['a'] < 0).astype(float) * (1 - cusp['a'].clip(-1, 0).abs())).rolling(5).mean()
print(f'Cusp parameters computed: {len(cusp)} windows')

---
# PART 6: INTEGRATED FRAGILITY INDEX (IFI)

In [ ]:
# === CONSTRUCT IFI ===
# Simple weighted combination
# Weights based on theoretical importance, not data fitting

def normalize(s):
    return (s - s.min()) / (s.max() - s.min() + 1e-8)

# Align indices
ifi_idx = F_t.index.intersection(cusp.index)

IFI = (
    0.35 * normalize(F_t.loc[ifi_idx]) +              # Latent fragility (from FA)
    0.25 * normalize(F_bands['slow'].loc[ifi_idx]) +   # Structural buildup
    0.15 * normalize(F_bands['medium'].loc[ifi_idx]) + # Resonance zone
    0.10 * normalize(F_bands['fast'].loc[ifi_idx]) +   # Fast noise
    0.15 * normalize(BIF_RISK.loc[ifi_idx])            # Bifurcation risk
)
IFI = IFI.dropna()

print(f'IFI computed: {len(IFI)} observations')
print(f'IFI range: [{IFI.min():.3f}, {IFI.max():.3f}]')

In [ ]:
# === THRESHOLDS ===
threshold_warning = IFI.quantile(0.75)
threshold_critical = IFI.quantile(0.95)

print(f'\nThresholds:')
print(f'  Warning (75th pct): {threshold_warning:.3f}')
print(f'  Critical (95th pct): {threshold_critical:.3f}')

---
# PART 7: STATISTICAL VALIDATION

In [ ]:
# === CREATE LABELS ===
def create_crisis_labels(index, crises, pre_window=20, post_window=10):
    labels = pd.Series(0, index=index)
    for name, date in crises.items():
        start = date - pd.Timedelta(days=pre_window)
        end = date + pd.Timedelta(days=post_window)
        labels[(labels.index >= start) & (labels.index <= end)] = 1
    return labels

y_true = create_crisis_labels(IFI.index, CRISES)
print(f'Crisis labels: {y_true.sum()} crisis days ({100*y_true.mean():.1f}%)')

In [ ]:
# === ROC/AUC COMPARISON ===
models = {
    'IFI (Integrated)': IFI,
    'CF Only': normalize(CF.loc[IFI.index]),
    'F_t (Factor Analysis)': normalize(F_t.loc[IFI.index]),
    'Volatility Only': normalize(ret.std(axis=1).rolling(20).mean().loc[IFI.index])
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC
ax1 = axes[0]
auc_results = {}
for name, score in models.items():
    score_aligned = score.dropna()
    y_aligned = y_true.loc[score_aligned.index]
    fpr, tpr, _ = roc_curve(y_aligned, score_aligned)
    roc_auc = auc(fpr, tpr)
    auc_results[name] = roc_auc
    ax1.plot(fpr, tpr, label=f'{name} (AUC={roc_auc:.3f})', linewidth=2)

ax1.plot([0, 1], [0, 1], 'k--')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('ROC Curves')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# AUC bars
ax2 = axes[1]
names = list(auc_results.keys())
aucs = list(auc_results.values())
colors = ['#10b981' if auc_results[n] == max(aucs) else '#6b7280' for n in names]
ax2.barh(names, aucs, color=colors)
ax2.axvline(0.5, color='red', linestyle='--')
ax2.set_xlabel('AUC')
ax2.set_xlim(0.4, 0.9)
ax2.set_title('AUC Comparison')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_final_roc.png', dpi=150)
plt.show()

print('\n=== AUC Results ===')
for name, score in sorted(auc_results.items(), key=lambda x: -x[1]):
    print(f'{name:25s}: {score:.3f}')

In [ ]:
# === PERMUTATION TEST ===
n_perm = 500
null_aucs = []

best_model = 'CF Only'
score = models[best_model].dropna()
y_test = y_true.loc[score.index]

# Observed
fpr, tpr, _ = roc_curve(y_test, score)
observed_auc = auc(fpr, tpr)

# Permutation
for _ in range(n_perm):
    y_perm = y_test.sample(frac=1).reset_index(drop=True)
    fpr, tpr, _ = roc_curve(y_perm.values, score.values)
    null_aucs.append(auc(fpr, tpr))

p_value = (np.array(null_aucs) >= observed_auc).mean()

plt.figure(figsize=(10, 4))
plt.hist(null_aucs, bins=30, alpha=0.7, color='gray', label='Null distribution')
plt.axvline(observed_auc, color='red', linewidth=2, label=f'Observed={observed_auc:.3f}')
plt.xlabel('AUC')
plt.ylabel('Frequency')
plt.title(f'Permutation Test: p = {p_value:.4f}')
plt.legend()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_final_permutation.png', dpi=150)
plt.show()

print(f'\nPermutation Test (n={n_perm}):')
print(f'  Observed AUC: {observed_auc:.4f}')
print(f'  Null mean: {np.mean(null_aucs):.4f}')
print(f'  p-value: {p_value:.4f}')
print(f'  Significant: {"YES ✓" if p_value < 0.05 else "NO"}')

---
# PART 8: FINAL VISUALIZATION

In [ ]:
# === MAIN FIGURE ===
fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)

# A. Crisis Factor (the star)
ax1 = axes[0]
ax1.fill_between(CF.index, CF.values, alpha=0.4, color='red')
ax1.plot(CF.index, CF.values, 'r-', linewidth=0.5)
ax1.axhline(CF.quantile(0.75), color='orange', linestyle='--', label='75th pct')
ax1.axhline(CF.quantile(0.95), color='darkred', linestyle='--', label='95th pct')
ax1.set_ylabel('CF')
ax1.set_title('A. Crisis Factor (AUC={:.3f})'.format(auc_results.get('CF Only', 0)))
ax1.legend(loc='upper right')

# B. Latent F_t
ax2 = axes[1]
ax2.fill_between(F_t.index, F_t.values, alpha=0.4, color='purple')
ax2.plot(F_t.index, F_t.values, 'purple', linewidth=0.5)
ax2.axhline(F_t.quantile(0.90), color='orange', linestyle='--')
ax2.set_ylabel('F_t')
ax2.set_title('B. Latent Fragility (Factor Analysis)')

# C. IFI
ax3 = axes[2]
ax3.fill_between(IFI.index, IFI.values, alpha=0.4, color='blue')
ax3.plot(IFI.index, IFI.values, 'b-', linewidth=0.5)
ax3.axhline(threshold_warning, color='orange', linestyle='--', label='Warning')
ax3.axhline(threshold_critical, color='darkred', linestyle='--', label='Critical')
ax3.set_ylabel('IFI')
ax3.set_title('C. Integrated Fragility Index')
ax3.legend(loc='upper right')

# D. Bifurcation status
ax4 = axes[3]
ax4.fill_between(cusp.index, cusp['a'], 0, 
                 where=cusp['a'] < 0, alpha=0.5, color='red', label='Bistable')
ax4.fill_between(cusp.index, cusp['a'], 0,
                 where=cusp['a'] >= 0, alpha=0.3, color='green', label='Stable')
ax4.axhline(0, color='black', linestyle='-', linewidth=0.5)
ax4.set_ylabel('Cusp a')
ax4.set_title('D. Bifurcation Status (a < 0 = critical regime)')
ax4.legend(loc='upper right')

# Add crisis lines
for ax in axes:
    for name, date in CRISES.items():
        if date > CF.index.min() and date < CF.index.max():
            ax.axvline(date, color='blue', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_final_summary.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# === LEAD TIME ANALYSIS ===
print('\n=== Pre-Crisis Levels and Lead Times ===')

def get_precisis_value(series, crisis_date, days_before=20):
    pre = series[(series.index < crisis_date) & 
                 (series.index > crisis_date - pd.Timedelta(days=days_before))]
    return pre.mean() if len(pre) > 0 else np.nan

results = []
for crisis_name, crisis_date in CRISES.items():
    if crisis_date < CF.index.min() + pd.Timedelta(days=200):
        continue
    if crisis_date > CF.index.max():
        continue
        
    cf_pre = get_precisis_value(CF, crisis_date)
    ifi_pre = get_precisis_value(IFI, crisis_date) if crisis_date in IFI.index or crisis_date > IFI.index.min() else np.nan
    
    results.append({
        'Crisis': crisis_name,
        'Date': crisis_date.date(),
        'CF_pre': cf_pre,
        'IFI_pre': ifi_pre
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

In [ ]:
# === EXPORT ===
import json
from datetime import datetime

# Current values
current = {
    'cf': float(CF.iloc[-1]),
    'f_t': float(F_t.iloc[-1]),
    'ifi': float(IFI.iloc[-1]),
    'bifurcation_stable': bool(cusp['a'].iloc[-1] >= 0)
}

# Determine status
if current['ifi'] >= threshold_critical:
    status = 'CRITICAL'
elif current['ifi'] >= threshold_warning:
    status = 'WARNING'
else:
    status = 'STABLE'

export = {
    'version': 'Great Caria Final (Simplified)',
    'generated': datetime.now().isoformat(),
    'methodology': 'CF + 3-Band Decomposition + Factor Analysis + Cusp Bifurcation',
    'status': status,
    'current': current,
    'thresholds': {
        'warning': float(threshold_warning),
        'critical': float(threshold_critical)
    },
    'factor_loadings': loadings.set_index('signal')['loading'].to_dict(),
    'auc_scores': auc_results,
    'permutation_test': {
        'p_value': float(p_value),
        'significant': p_value < 0.05
    },
    'crises_validated': len(results_df)
}

with open('/content/drive/MyDrive/Caria/great_caria_final.json', 'w') as f:
    json.dump(export, f, indent=2)

print('\n✓ Exported: great_caria_final.json')
print(json.dumps(export, indent=2))

In [ ]:
# === FINAL SUMMARY ===
print('\n' + '='*70)
print('GREAT CARIA FINAL - SUMMARY')
print('='*70)

print('\n📊 MODEL COMPONENTS:')
print('  1. Crisis Factor (CF) = correlation × volatility')
print('  2. Factor Analysis → Latent F_t')
print('  3. 3-Band Decomposition (Slow/Medium/Fast)')
print('  4. Cusp Bifurcation Detection')

print('\n📈 PERFORMANCE:')
for name, score in sorted(auc_results.items(), key=lambda x: -x[1]):
    star = '⭐' if score == max(auc_results.values()) else '  '
    print(f'  {star} {name:25s}: AUC = {score:.3f}')

print(f'\n🔬 SIGNAL AUTHENTICITY:')
print(f'  Permutation test p-value: {p_value:.4f}')
print(f'  Result: {"REAL SIGNAL ✓" if p_value < 0.05 else "Not significant"}')

print(f'\n⚠️ CURRENT STATUS: {status}')
print(f'  IFI: {current["ifi"]:.3f}')
print(f'  Bifurcation: {"Stable" if current["bifurcation_stable"] else "BISTABLE"}')

print('\n' + '='*70)